In [1]:
import os

In [2]:
%pwd

'/Users/sneha/MLOps/research'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    
    local_data_file: Path
    unzip_dir: Path

In [10]:
import sys, os
sys.path.append(os.path.abspath("../src"))


In [11]:
import sys
print(sys.path[-1])  # last added path
!ls ../src


/Users/sneha/MLOps/src
cnnClassifier


In [15]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [22]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.entity import DataIngestionConfig


ImportError: cannot import name 'CONFIG_FILE_PATH' from 'cnnClassifier.constants' (/Users/sneha/MLOps/src/cnnClassifier/constants/__init__.py)

In [23]:
from pathlib import Path
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")


In [24]:
import cnnClassifier.constants
print(cnnClassifier.constants.__file__)


/Users/sneha/MLOps/src/cnnClassifier/constants/__init__.py


In [25]:
import cnnClassifier.constants as const
print(dir(const))


['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [26]:
print(const.__file__)


/Users/sneha/MLOps/src/cnnClassifier/constants/__init__.py


In [27]:
import importlib
import cnnClassifier.constants as const
importlib.reload(const)

print(dir(const))  # should now include CONFIG_FILE_PATH, PARAMS_FILE_PATH


['CONFIG_FILE_PATH', 'PARAMS_FILE_PATH', 'Path', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [34]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

from pathlib import Path



In [49]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config=config

    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''
        try:
            dataset_url = self.config.source_URL

            zip_download_dir= self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id=dataset_url.split("/")[-2]
            prefix= 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [50]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-17 21:19:56,588:INFO:common:yaml file: config/config.yaml loaded successfully]
[2025-08-17 21:19:56,590:INFO:common:yaml file: params.yaml loaded successfully]
[2025-08-17 21:19:56,591:INFO:common:created directory at: artifacts]
[2025-08-17 21:19:56,592:INFO:common:created directory at: artifacts/data_ingestion]
[2025-08-17 21:19:56,592:INFO:4011944139:Downloading data from https://drive.google.com/file/d/1XeIX-HtiFbw47xyCqaufKl2v-CVecRZ2/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1XeIX-HtiFbw47xyCqaufKl2v-CVecRZ2
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1XeIX-HtiFbw47xyCqaufKl2v-CVecRZ2&confirm=t&uuid=28dcb326-5bff-43e0-b7e3-ec63551564f9
To: /Users/sneha/MLOps/artifacts/data_ingestion/data.zip
100%|██████████| 57.8M/57.8M [00:13<00:00, 4.27MB/s]

[2025-08-17 21:20:13,709:INFO:4011944139:Downloaded data from https://drive.google.com/file/d/1XeIX-HtiFbw47xyCqaufKl2v-CVecRZ2/view?usp=sharing into file artifacts/data_ingestion/data.zip]


In [32]:
import os
os.chdir("..")  # if running from research/
print(os.getcwd())  # should show MLOPS


/Users/sneha/MLOps


In [36]:
with open("config/config.yaml", "r") as f:
    print(f.read())


artifacts_root: artifacts

data_ingestion:
  root_dir: artifacts/data_ingestion
  source_URL: "https://drive.google.com/file/d/1XeIX-HtiFbw47xyCqaufKl2v-CVecRZ2/view?usp=sharing"
  local_data_file: artifacts/data_ingestion/data.zip
  unzip_dir: artifacts/data_ingestion


In [37]:
from cnnClassifier.utils.common import read_yaml
from cnnClassifier.constants import CONFIG_FILE_PATH

print(CONFIG_FILE_PATH.resolve())  # see absolute path


/Users/sneha/MLOps/config/config.yaml


In [42]:
from pathlib import Path

yaml_path = Path("config/config.yaml")
print("YAML file path:", yaml_path)
print("Exists?", yaml_path.exists())
print("Size:", yaml_path.stat().st_size if yaml_path.exists() else "File not found")


YAML file path: config/config.yaml
Exists? True
Size: 270


In [43]:
import yaml

with open("config/config.yaml", "r") as f:
    config_data = yaml.safe_load(f)

print(config_data)


{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1XeIX-HtiFbw47xyCqaufKl2v-CVecRZ2/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
